### Carga de librerias

In [5]:
import pandas as pd
import glob
import re
import unicodedata
from rapidfuzz import fuzz


### Carga de archivos y visualización

In [6]:
"""
Carga y combina múltiples archivos CSV desde una carpeta específica,
saltando filas de encabezado no necesarias y líneas de pie de página.
"""

ruta = "mineduc/*.csv"

# Leer y procesar cada archivo CSV encontrado en la ruta:

dfs = [
    pd.read_csv(archivo, skiprows=25, skipfooter=5, engine="python")
    for archivo in sorted(glob.glob(ruta))
]

df_final = pd.concat(dfs, ignore_index=True)
print(df_final.shape)


(6600, 17)


In [7]:
df_final.head()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6600 entries, 0 to 6599
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CODIGO           6600 non-null   object
 1   DISTRITO         6600 non-null   object
 2   DEPARTAMENTO     6600 non-null   object
 3   MUNICIPIO        6600 non-null   object
 4   ESTABLECIMIENTO  6600 non-null   object
 5   DIRECCION        6598 non-null   object
 6   TELEFONO         6554 non-null   object
 7   SUPERVISOR       6600 non-null   object
 8   DIRECTOR         6574 non-null   object
 9   NIVEL            6600 non-null   object
 10  SECTOR           6600 non-null   object
 11  AREA             6600 non-null   object
 12  STATUS           6600 non-null   object
 13  MODALIDAD        6600 non-null   object
 14  JORNADA          6600 non-null   object
 15  PLAN             6600 non-null   object
 16  DEPARTAMENTAL    6600 non-null   object
dtypes: object(17)
memory usage: 876.7

### Formato CODIGO

In [9]:
"""
Verificación del formato de la columna 'CODIGO' en el DataFrame `df_final`.
"""
pattern = re.compile(r"^\d{2}-\d{2}-\d{4}-\d{2}$")
filas_incorrectas = df_final[~df_final["CODIGO"].astype(str).apply(lambda x: bool(pattern.match(x)))]
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


### Formato DISTRITO 

In [11]:
"""
Verificación del formato de la columna 'DISTRITO' en el DataFrame `df_final`.
"""
pattern = re.compile(r"^\d{2}-\d{3}$")
filas_incorrectas = df_final[~df_final["DISTRITO"].astype(str).apply(lambda x: bool(pattern.match(x)))]
print(filas_incorrectas)


Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


### Formato ESTABLECIMIENTO

In [12]:
"""
Verificación de mayúsculas en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.
"""
filas_incorrectas = df_final[df_final["ESTABLECIMIENTO"] != df_final["ESTABLECIMIENTO"].str.upper()]
print(filas_incorrectas)



Empty DataFrame
Columns: [CODIGO, DISTRITO, DEPARTAMENTO, MUNICIPIO, ESTABLECIMIENTO, DIRECCION, TELEFONO, SUPERVISOR, DIRECTOR, NIVEL, SECTOR, AREA, STATUS, MODALIDAD, JORNADA, PLAN, DEPARTAMENTAL]
Index: []


In [13]:

"""
Reemplazo de comillas simples por comillas dobles en la columna 'ESTABLECIMIENTO'
del DataFrame `df_final`, sin afectar las comillas que forman parte de una palabra.
"""

df_final["ESTABLECIMIENTO"] = (
    df_final["ESTABLECIMIENTO"]
    .astype(str)  # Asegurar que todos los valores sean strings
    .str.replace(r"(?<!\w)'|'(?!\w)", '"', regex=True)  # Reemplazo condicional
)


In [15]:
"""
Detección de valores repetidos en la columna 'ESTABLECIMIENTO' del DataFrame `df_final`.
"""

repetidos = df_final[df_final["ESTABLECIMIENTO"].duplicated(keep=False)]
print(repetidos.sort_values("ESTABLECIMIENTO"))



             CODIGO DISTRITO   DEPARTAMENTO      MUNICIPIO  \
4321  17-05-0335-46   17-007          PETEN    LA LIBERTAD   
4292  17-05-0211-46   17-007          PETEN    LA LIBERTAD   
4315  17-05-0319-46   17-007          PETEN    LA LIBERTAD   
4304  17-05-0260-46   17-007          PETEN    LA LIBERTAD   
5947  06-01-0118-46   06-039     SANTA ROSA        CUILAPA   
...             ...      ...            ...            ...   
392   04-01-0058-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
525   04-01-3245-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
520   04-01-3068-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
431   04-01-0230-46   04-001  CHIMALTENANGO  CHIMALTENANGO   
473   04-01-0357-46   04-001  CHIMALTENANGO  CHIMALTENANGO   

                                        ESTABLECIMIENTO  \
4321          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4292          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4315          "CENTRO DE EDUCACION INTEGRAL EL NARANJO"   
4304          "CENT

In [16]:
"""
Detección de nombres de establecimientos con alta similitud (mayor al 95%)
utilizando la librería RapidFuzz.
"""
nombres = df_final["ESTABLECIMIENTO"].unique()
resultados = []
for i, nombre in enumerate(nombres):
    for j in range(i + 1, len(nombres)):  
        similitud = fuzz.ratio(nombre, nombres[j])  
        if similitud > 95 and similitud < 100:      
            resultados.append({
                "Nombre_Original": nombre,
                "Nombre_Comparado": nombres[j],
                "Similitud (%)": similitud
            })

df_parecidos = pd.DataFrame(resultados).sort_values("Similitud (%)", ascending=False)
print(f"Total de pares con similitud > 95%: {len(df_parecidos)}")
df_parecidos


Total de pares con similitud > 95%: 879


,Nombre_Original,Nombre_Comparado,Similitud (%)
295,"COLEGIO ""CENTRO DE FORMACION PROFESIONAL PRE-U...","COLEGIO ""CENTRO DE FORMACION PROFESIONAL PREUN...",99.393939
527,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,COLEGIO EDUCATIVO PRIVADO MIXTO DE FORMACION I...,99.363057
214,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACIÓN DIVERSIFICADA ...,99.328859
328,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,INSTITUTO MIXTO MUNICIPAL DE EDUCACIÓN BÁSICA ...,99.310345
87,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,CENTRO DE ESTUDIOS TECNICOS Y AVANZADOS DE CHI...,99.290780
...,...,...,...
95,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA, PR...",PROGRAMA NACIONAL DE EDUCACION ALTERNATIVA -PR...,95.049505
658,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA ...,INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADO-...,95.049505
345,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA , P...",PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,95.049505
739,PROGRAMA NACIONA DE EDUCACIÓN ALTERNATIVA -PRO...,"PROGRAMA NACIONAL DE EDUCACIÓN ALTERNATIVA, PR...",95.049505


In [17]:
def limpiar_establecimiento(df, columna="ESTABLECIMIENTO"):
    """
    Limpia y normaliza los valores de una columna de nombres de establecimientos educativos.
    """
    def normalizar_tildes(texto):
        return unicodedata.normalize("NFC", texto)


    correcciones_seguras = {
        "INSDUSTRIAL": "INDUSTRIAL",
        "BILINGUE": "BILINGÜE"
    }

    correcciones_tildes = {
        "BILINGÚE": "BILINGÜE",
        "BINLINGÜE": "BILINGÜE",
        "TECNOLGÍA": "TECNOLOGÍA",
        "TECNOLOGIA": "TECNOLOGÍA",
        "TECNÓLOGICO": "TECNOLÓGICO",
        "EDCACIÓN": "EDUCACIÓN",
        "JÉSUS": "JESÚS",
        "EVANGELICO": "EVANGÉLICO",
        "COMPUTACION": "COMPUTACIÓN",
        "INFORMATICA": "INFORMÁTICA",
        "TECNICAS": "TÉCNICAS",
        "FORMACION": "FORMACIÓN",
        "BASICA": "BÁSICA"
    }


    df[columna] = (
        df[columna]
        .astype(str)                                
        .str.upper()                                
        .apply(normalizar_tildes)                   
        .str.replace(r"[-_]", " ", regex=True)      
        .str.replace(r"\s+", " ", regex=True)       
        .str.strip()                                
        .str.rstrip(".")                            
        .str.replace(r'^"+', '', regex=True)        
        .str.replace(r'"+$', '', regex=True)        
        .str.replace(r'""', '"', regex=True)        
        .str.strip()
        .str.replace(r'" +', '"', regex=True)       
        .str.replace(r' +"', '"', regex=True)       
        .str.replace(r"\(\s*(.*?)\s*\)", r"(\1)", regex=True)  
        .replace(correcciones_seguras, regex=True)  
        .replace(correcciones_tildes, regex=True)   
        .str.replace(r"(\d),(\d{3})", r"\1\2", regex=True)      
        .str.replace(r"\bNO\.?\s*(\d+)", r"NO. \1", regex=True) 
        .str.replace(r"(NO\. \d+),", r"\1, ", regex=True)       
        .str.replace(r"\s+,", ",", regex=True)      
        .str.replace(r",\s*", ", ", regex=True)     
        .str.replace(r"\s+", " ", regex=True)       
        .str.strip()
    )

    return df

df_final = limpiar_establecimiento(df_final, "ESTABLECIMIENTO")

# Supervisor/director

In [20]:
def normalize_nfc(s: str) -> str:
    return unicodedata.normalize("NFC", s)

def strip_diacritics(s: str) -> str:
    s = unicodedata.normalize("NFD", s)
    s = "".join(c for c in s if unicodedata.category(c) != "Mn")
    return unicodedata.normalize("NFC", s)

def limpiar_nombre_persona(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = normalize_nfc(s).upper().strip()
    s = re.sub(r"[“”]", '"', s)
    s = re.sub(r"[‘’]", "'", s)
    s = re.sub(r'^"+|"+$', "", s)           
    s = re.sub(r"[^\w\s'.-]", " ", s)       
    s = re.sub(r"\s+", " ", s).strip()

    s = re.sub(r"\b(D|DEL|DE LA|DE LOS|DE LAS|DE)\b", lambda m: m.group(0), s)

    s = re.sub(r"\.(?=\S)", ". ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def clave_bloqueo(s: str) -> str:
    s2 = strip_diacritics(s)
    s2 = re.sub(r"[^A-Z\s]", " ", s2.upper())
    toks = sorted(set(s2.split()))
    return " ".join(toks)

def iniciales(s: str) -> str:
    return "".join(t[0] for t in s.split() if t and t[0].isalpha())

# Aplicar limpieza a SUPERVISOR / DIRECTOR
for col in ["SUPERVISOR", "DIRECTOR"]:
    if col in df_final.columns:
        df_final[col + "_STD"] = df_final[col].apply(limpiar_nombre_persona)
        df_final[col + "_BLOCK"] = df_final[col + "_STD"].apply(clave_bloqueo)
        df_final[col + "_INI"] = df_final[col + "_STD"].apply(iniciales)

def construir_mapeo_canon(df_col_std: pd.Series, df_col_block: pd.Series, umbral=95):
    """
    Devuelve dict: nombre_std -> nombre_canónico
    """
    mapping = {}
    tmp = pd.DataFrame({
        "STD": df_col_std,
        "BLOCK": df_col_block
    }).dropna().drop_duplicates()

    for block, grp in tmp.groupby("BLOCK"):
        vals = grp["STD"].tolist()
        if len(vals) == 1:
            mapping[vals[0]] = vals[0]
            continue


        clusters = []  
        for v in vals:
            asignado = False
            for cl in clusters:
                if any(fuzz.token_set_ratio(v, w) >= umbral for w in cl):
                    cl.append(v)
                    asignado = True
                    break
            if not asignado:
                clusters.append([v])

        freqs = df_col_std.value_counts()
        for cl in clusters:
            canon = max(cl, key=lambda x: freqs.get(x, 0))
            for v in cl:
                mapping[v] = canon
    return mapping

for col in ["SUPERVISOR", "DIRECTOR"]:
    if col + "_STD" in df_final.columns:
        mapeo = construir_mapeo_canon(df_final[col + "_STD"], df_final[col + "_BLOCK"], umbral=95)
        df_final[col + "_CANON"] = df_final[col + "_STD"].map(mapeo)
        n_variantes = (df_final[col + "_STD"] != df_final[col + "_CANON"]).sum()
        print(f"[{col}] variantes unificadas por similitud >=95: {n_variantes}")


[SUPERVISOR] variantes unificadas por similitud >=95: 13
[DIRECTOR] variantes unificadas por similitud >=95: 30


# Plan

In [21]:
def normalizar_plan_base(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = unicodedata.normalize("NFC", s).upper()
    s = re.sub(r"\s*\((.*?)\)\s*", r" \1 ", s)  
    s = re.sub(r"[“”]", '"', s)
    s = re.sub(r"[‘’]", "'", s)
    s = s.replace("PLAN ", " ")
    s = re.sub(r"[^A-Z0-9\s\-]", " ", s)      
    s = re.sub(r"\s+", " ", s).strip()
    return s

PLAN_EQUIV = {
    "BASICO": "BÁSICO",
    "BASICA": "BÁSICO",
    "EDUCACION BASICA": "BÁSICO",
    "EDUCACIÓN BASICA": "BÁSICO",
    "EDUCACIÓN BÁSICA": "BÁSICO",

    "DIVERSIFICADO": "DIVERSIFICADO",
    "EDUCACION DIVERSIFICADA": "DIVERSIFICADO",
    "EDUCACIÓN DIVERSIFICADA": "DIVERSIFICADO",

    "PREPRIMARIA": "PREPRIMARIA",
    "PRIMARIA": "PRIMARIA",
    "CICLO BASICO": "BÁSICO",
    "CICLO BÁSICO": "BÁSICO",

    "TECNICO": "TÉCNICO",
    "TÉCNICO": "TÉCNICO",
    "TECNICO VOCACIONAL": "TÉCNICO VOCACIONAL",
    "TÉCNICO VOCACIONAL": "TÉCNICO VOCACIONAL",
    "PERITO CONTADOR": "PERITO CONTADOR",
    "BACHILLERATO EN CIENCIAS Y LETRAS": "BACHILLERATO EN CIENCIAS Y LETRAS",
    "MAGISTERIO": "MAGISTERIO",
}

RE_ACENTOS = {
    r"\bBASICO\b": "BÁSICO",
    r"\bTECNICO\b": "TÉCNICO",
}

def estandarizar_plan(s: str) -> str:
    base = normalizar_plan_base(s)

    std = PLAN_EQUIV.get(base)
    if std:
        return std

    if re.search(r"\b(BASICO|BÁSICO|CICLO BASICO|CICLO BÁSICO)\b", base):
        std = "BÁSICO"
    elif re.search(r"\bDIVERSIFICAD", base):
        std = "DIVERSIFICADO"
    elif re.search(r"\bPREPRIMARIA\b", base):
        std = "PREPRIMARIA"
    elif re.search(r"\bPRIMARIA\b", base):
        std = "PRIMARIA"
    elif re.search(r"\bTECNIC", base):
        std = "TÉCNICO"
    elif re.search(r"\bBACHILLER", base):
        std = "BACHILLERATO EN CIENCIAS Y LETRAS"
    elif re.search(r"\bPERITO\s+CONTADOR\b", base):
        std = "PERITO CONTADOR"
    elif re.search(r"\bMAGISTERIO\b", base):
        std = "MAGISTERIO"
    else:
        std = base  

    for pat, rep in RE_ACENTOS.items():
        std = re.sub(pat, rep, std)

    return std

if "PLAN" in df_final.columns:
    df_final["PLAN_STD"] = df_final["PLAN"].apply(estandarizar_plan)

    print("Top 30 categorías PLAN originales normalizadas (previas a mapping):")
    print(df_final["PLAN"].astype(str).str.upper().str.strip().value_counts().head(30))
    print("\nTop 30 categorías PLAN estandarizadas:")
    print(df_final["PLAN_STD"].value_counts().head(30))

Top 30 categorías PLAN originales normalizadas (previas a mapping):
PLAN
DIARIO(REGULAR)                          4052
FIN DE SEMANA                            1310
SEMIPRESENCIAL (FIN DE SEMANA)            490
SEMIPRESENCIAL (UN DÍA A LA SEMANA)       409
A DISTANCIA                               116
SEMIPRESENCIAL                             99
SEMIPRESENCIAL (DOS DÍAS A LA SEMANA)      62
VIRTUAL A DISTANCIA                        52
SABATINO                                    5
INTERCALADO                                 2
DOMINICAL                                   2
MIXTO                                       1
Name: count, dtype: int64

Top 30 categorías PLAN estandarizadas:
PLAN_STD
DIARIO REGULAR                         4052
FIN DE SEMANA                          1310
SEMIPRESENCIAL FIN DE SEMANA            490
SEMIPRESENCIAL UN D A A LA SEMANA       409
A DISTANCIA                             116
SEMIPRESENCIAL                           99
SEMIPRESENCIAL DOS D AS A LA SEMANA 